In [1]:
%pip install python-dotenv transformers accelerate bitsandbytes --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv
import torch
import os

In [4]:
# Load Environment Variables
load_dotenv()

# Hugging Face Token
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [5]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", token=HUGGING_FACE_TOKEN)

# Load model in 4-bit precision using bitsandbytes
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    token=HUGGING_FACE_TOKEN
)

# Create pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample = True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
# Define your prompt and generate activity suggestions function
def generate_activity_suggestions(mood, weather, mental_body_state, preferences):
    prompt = (
        f"[INST] The user is feeling {mood}, the weather is {weather}, and they are currently in a state of {mental_body_state}. "
        f"They typically enjoy activities like {preferences}. "
        f"Based on the weather, user's mood, state, and preferences, suggest at most 3 relevant activities "
        f"with a 50-word short description for each one, that would produce a healthy mental well-being in the user. [/INST]"
    )
    result = generator(prompt)
    return result[0]["generated_text"]

In [8]:
# Example Input
mood = "anxious"
weather = "sunny"
state = "HEALTHY_CONSUMPTION, GOOD_SLEEP, GOOD_HEART_HEALTH, GOOD_BlOOD_PRESSURE, ACTIVE, HIGH_Stress, LOW_Workload, HIGH_SOCIAL"
preferences = "socializing, listening to music, playing video games, running, hiking"

output = generate_activity_suggestions(mood, weather, state, preferences)
print(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] The user is feeling anxious, the weather is sunny, and they are currently in a state of HEALTHY_CONSUMPTION, GOOD_SLEEP, GOOD_HEART_HEALTH, GOOD_BlOOD_PRESSURE, ACTIVE, HIGH_Stress, LOW_Workload, HIGH_SOCIAL. They typically enjoy activities like socializing, listening to music, playing video games, running, hiking. Based on the weather, user's mood, state, and preferences, suggest at most 3 relevant activities with a 50-word short description for each one, that would produce a healthy mental well-being in the user. [/INST] 1. Socializing: Spend time with friends or family members outside, enjoying the beautiful day.
2. Listening to Music: Take a break from work or other responsibilities and listen to your favorite tunes.
3. Running: Go for a jog or run around the neighborhood, getting some exercise and fresh air.
